<a href="https://colab.research.google.com/github/OdysseusPolymetis/journees_cluster5b_7/blob/main/3_nlp_lat_gk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tokéniser, lemmatiser, étiqueter en latin et en grec**

---



## De quoi s'agit-il ?
Ici on va faire des expériences sur ces trois points, nécessaires pour le traitement statistique notamment, la **tokénisation**, la **lemmatisation** et l'**étiquetage syntaxique**.

## Quelques petits outils utiles pour les non-programmeurs
Cette liste est bien sûr non exhaustive, mais d'expérience je pense qu'elle peut servir comme pis aller.

##**Quelques outils en ligne**

###**UDPipe**
Vous le trouverez [ici](https://lindat.mff.cuni.cz/services/udpipe/).
<br>Vous pouvez l'utiliser pour des textes courts.

###**Deucalion**
Vous le trouverez [ici](https://dh.chartes.psl.eu/deucalion/).
<br>Je ne sais pas si c'est toujours maintenu, mais le modèle est correct, et l'usage facile.

###**VoyantTools**
Vous le trouverez [ici](https://voyant-tools.org/).
<br>C'est un outil uniquement pour la visualisation, mais on peut faire des choses intéressantes.

##**Tokénisation, lemmatisation et postagging avec `stanza`**

On va commencer par tester **`stanza`**. Il y a de très nombreux modules qui existent (comme`spacy` et `pie-extended`), mais `stanza` est assez facile à utiliser (je trouve), et propose de très nombreuses langues (et surtout de très nombreux modèles par langue (4 pour le latin par exemple)).

In [ ]:
!pip install stanza

Let's try on a simple string first, which is encapsulated in the `catilinaires` variable.

In [ ]:
catilinaires="Quousque tandem abutere, Catilina, patientia nostra ? Quamdiu etiam furor iste tuus nos eludet ? Quem ad finem sese effrenata jactabit audacia ? Nihilne te nocturnum praesidium Palatii, nihil urbis vigiliae, nihil timor populi, nihil concursus bonorum omnium, nihil hic munitissimus habendi senatus locus, nihil horum ora vultusque moverunt ? Patere tua consilia non sentis ? Constrictam jam horum omnium scientia teneri conjurationem tuam non vides ? Quid proxima, quid superiore nocte egeris, ubi fueris, quos convocaveris, quid consilii ceperis, quem nostrum ignorare arbitraris ? O tempora ! O mores ! Senatus haec intellegit, consul videt. Hic tamen vivit."

###**stanza**

Pourquoi je parle de `stanza` ici ? Parce que c'est effectivement de l'IA, mais pas forcément comme on l'entend d'habitude. C'est de l'apprentissage profond, mais les modèles ne sont pas des transformers. Ce sont des architectures plus légères (des encodeurs Bi-LSTM), entraînées sur des textes déjà étiquetés. L'avantage, c'est que c'est moins coûteux sur le plan computationnel, et plus contrôlable. En revanche, c'est plus strict et donc moins enclin à la généralisation ou au hors domaine.

`stanza` dispose de nombreux modèles (voilà une [liste](https://stanfordnlp.github.io/stanza/performance.html)), que vous pouvez utiliser en appelant le code langue, comme `grc` pour le grec ancien ou `la` pour le latin. Mais vous pouvez aussi être plus précis sur le modèle que vous souhaitez en particulier.

In [ ]:
import stanza
stanza.download('la', package="perseus")

On commence par construire une pipeline, pour dire ce que l'on voudra utiliser (on ne prend pas le `ner` par exemple).

In [ ]:
nlp_stanza = stanza.Pipeline(lang='la', package="perseus", processors='tokenize,pos,lemma, depparse')

Ici la variable `catilinaires_analyzed` est un objet `stanza`, où sont encapsulées toutes les infos générées par le moteur.

In [ ]:
catilinaires_analyzed=nlp_stanza(catilinaires)

In [ ]:
type(catilinaires_analyzed)

Voilà quelques résultats, avec du découpage en phrases, et un extrait des étiquettes obtenues.

`stanza` divise le texte en phrases, puis chaque phrase en une liste de tokens, qui ont tous des attributs type `.lemma`, ou `.pos`.

In [ ]:
for sent in catilinaires_analyzed.sentences:
  print("XXXXX "+sent.text+" XXXXX")

In [ ]:
for sent in catilinaires_analyzed.sentences:
  for token in sent.words:
    print(token.text + ' - ' + token.lemma + ' - ' + token.pos)

Essayons maintenant avec un beaucoup plus gros texte.

Ici je vous propose par défaut le texte de l'Odyssée, mais vous pouvez tester avec n'importe quel autre `.txt`, veillez simplement à ne pas avoir de caractères hors unicode, et un texte en format plain text.

In [ ]:
import stanza
stanza.download('grc')
import string

Prenons _L'Odyssée_.

In [ ]:
!wget https://raw.githubusercontent.com/OdysseusPolymetis/digital_classics_course/refs/heads/main/odyssee_integrale.txt

In [ ]:
!wget https://raw.githubusercontent.com/OdysseusPolymetis/digital_classics_course/refs/heads/main/stopwords_gk.txt
!wget https://raw.githubusercontent.com/OdysseusPolymetis/digital_classics_course/refs/heads/main/stopwords_lat.txt

Pour vos propres projets, vous pouvez trouver des listes de mots outils [ici](https://github.com/stopwords-iso). Moi je vais utiliser une version custom par défaut, mais c'est le même principe. Je vous propose donc deux listes custom, il vous suffit de changer le chemin d'accès au fichier en fonction de votre choix de langue (il suffit de changer `gk` en `lat` dans la cellule suivante).

In [ ]:
stopwords = open("/content/stopwords_gk.txt",'r',encoding="utf8").read().split("\n")

L'avantage d'utiliser des listes de mots outils inscrits dans un fichier est que vous pouvez ajouter vos propres mots en cliquant ici sur la gauche sur l'icône dossier (📁).
<br>Si vous cliquez accidentellement (après avoir cliqué sur l'icône dossier) sur la seconde icône dossier (celle avec ".."), ne vous inquiétez pas : le dossier originel est en fait le dossier "📁 `content`", que vous pouvez aussi ouvrir.

Vous pouvez donc mettre votre propre texte en appuyant une fois sur le dossier, et vous pouvez faire un "drag and drop" de votre fichier. Évitez, même comme règle générale, les espaces et les accents dans les fichiers et dossiers.

Une fois que vous avez mis votre texte, plusieurs choses à vérifier.

*   D'abord, changer le chemin vers votre fichier dans la cellule ci-dessous.
*   Ensuite, vérifier le code langue de votre moteur, dans cette cellule :

```python
nlp_stanza = stanza.Pipeline(lang='fr', etc.)
```



In [ ]:
filepath_of_text = "/content/odyssee_integrale.txt"

In [ ]:
full_text = open(filepath_of_text, encoding="utf-8").read()

In [ ]:
nlp_stanza = stanza.Pipeline(lang='grc', processors='tokenize,pos,lemma')

La fonction qui suit permet simplement de mieux gérer la mémoire.

In [ ]:
def batch_process(text, nlp, batch_size=100):
    paragraphs = text.split('\n')
    batches = [paragraphs[i:i + batch_size] for i in range(0, len(paragraphs), batch_size)]

    words = []

    for batch in batches:
        batch_text = '\n'.join(batch)
        doc = nlp(batch_text)
        for sentence in doc.sentences:
            for word in sentence.words:
                token={}
                if word.lemma is not None:
                    token["word"]=word.text
                    token["lemma"]=word.lemma
                    token["pos"]=word.pos
                    words.append(token)

    return words

C'est la cellule suivante qui va prendre du temps.

In [ ]:
odyssey = batch_process(full_text, nlp_stanza)

In [ ]:
print(odyssey[15:25])

Dans la cellule suivante, on va prendre tous les tokens issus de l'analyse, et on va les stocker dans trois listes, `forms`, `lemmas` and `no_stop`. Dans la liste `forms`, on garde simplement les mots tels quels (le texte lui-même donc). Dans la liste `lemmas`, on met tous les lemmes du texte. Enfin, dans la liste `no_stop`, on met tous les lemmes sans mots outils et sans ponctuation.

On va ensuite pouvoir utiliser ces listes pour une expérience toute simple (qui montre l'utilité d'un pré-traitement correct).

In [ ]:
forms = []
lemmas = []
no_stop = []

for token in miserables_analyzed:
    form = token["word"]
    lemma = token["lemma"]

    if lemma not in string.punctuation:
        forms.append(form)
        lemmas.append(lemma)

    if lemma not in string.punctuation and lemma not in stopwords:
        no_stop.append(lemma)

Deux cellules de vérification par la suite, vous devez obtenir plus que 0.

In [ ]:
len(lemmas)

In [ ]:
len(no_stop)

Ici pas la peine de comprendre en détail, gardez juste à l'idée que la fonction qui suit crée un nuage de mots, qui va varier en fonction de la liste qu'on va lui passer en paramètre. Les mots les plus importants (fréquence relative) apparaissent en gros.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

def create_word_cloud(words_list, title):
    text = ' '.join(words_list)

    radius = 495

    diameter = radius * 2
    center = radius
    x, y = np.ogrid[:diameter, :diameter]
    mask = (x - center) ** 2 + (y - center) ** 2 > radius ** 2
    mask = 255 * mask.astype(int)

    mask_rgba = np.dstack((mask, mask, mask, 255 - mask))

    wordcloud = WordCloud(repeat=False, width=diameter, height=diameter,
                          background_color=None, mode="RGBA", colormap='plasma',
                          mask=mask_rgba).generate(text)

    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
create_word_cloud(forms, 'Word Cloud for Forms')

In [ ]:
create_word_cloud(lemmas, 'Word Cloud for Lemmas')

In [ ]:
create_word_cloud(no_stop, 'Word Cloud for Lemmas without stopwords')

J'ai ajouté une petite cellule qui vous permet de télécharger les différentes listes dans des fichiers (par exemple si vous voulez les mettre dans Voyant Tools ou autre).

In [ ]:
with open("/content/forms.txt", "w", encoding="utf8") as f, open("/content/lemmas.txt", "w", encoding="utf8") as f2, open("/content/pullito.txt", "w", encoding="utf8") as f3:
    f.write("\n".join(forms))
    f2.write("\n".join(lemmas))
    f3.write("\n".join(no_stop))

## **Avec les `transformers`**

Plusieurs avantages aux transformers:


*   ils sont meilleurs pour le hors domaine (parce qu'ils sont entraînés sur plein de langues en même temps, ici par exemple sur du xlm-roberta), et donc potentiellement plus solides,
*   ils gèrent donc mieux les mots qu'ils ne connaissent pas, ainsi que les formes rares,
*   ils sont assez faciles à implémenter, et surtout à affiner (sur des données personnalisées)
<br>Le problème c'est qu'ils restent lourds et coûteux.





In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
name = "wietsedv/xlm-roberta-base-ft-udpos28-grc"
tok = AutoTokenizer.from_pretrained(name)
mdl = AutoModelForTokenClassification.from_pretrained(name)
pos = pipeline("token-classification", model=mdl, tokenizer=tok, aggregation_strategy="simple")
print(pos("ἀνὴρ σοφός ἐστι."))

Et voici une petite démonstration de pourquoi on appelle ça un transformer (enfin pas vraiment, mais c'est l'idée) : le traitement simultané en plusieurs langues

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
name = "jordigonzm/mdeberta-v3-base-multilingual-pos-tagger"
tok = AutoTokenizer.from_pretrained(name)
mdl = AutoModelForTokenClassification.from_pretrained(name)
pos = pipeline("token-classification", model=mdl, tokenizer=tok, aggregation_strategy="simple")
print(pos("ἀνὴρ σοφός ἐστι."))
print(pos("Aujourd'hui, maman est morte. Ou peut-être hier, je ne sais pas."))